In [38]:
import pandas as pd

In [39]:
true_train = pd.read_csv("/mnt/efs/shivamsinghal/RAPL/raw_data/finalized_true_dataset_one_annotator.csv")

In [40]:
true_train["time_spent_norm"] = (
    (true_train["time_spent"] - true_train["time_spent"].min())
    / (true_train["time_spent"].max() - true_train["time_spent"].min())
)

In [41]:
true_train["confidence_scores"] = (true_train["likert"].sub(4.5).abs() / 3.5)

In [42]:
lie_train = pd.read_csv("/mnt/efs/shivamsinghal/reliability-aware-preference-learning/RAPL/datasets/scale_LIE/final_scale_lie_train.csv")

In [58]:
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import log_loss

conf_iso = IsotonicRegression(out_of_bounds="clip") 
conf_iso.fit(true_train.confidence_scores, true_train.accuracy)

lie_train["TRUE_calibrated_confidence"] = conf_iso.predict(lie_train.confidence_scores)
raw_logloss  = log_loss(lie_train.correct_chosen, lie_train.confidence_scores)
cal_logloss  = log_loss(lie_train.correct_chosen, lie_train.TRUE_calibrated_confidence)
print(f"raw LL={raw_logloss:.3f} → calibrated LL={cal_logloss:.3f}")

raw LL=7.831 → calibrated LL=0.897


In [59]:
dec_iso = IsotonicRegression(out_of_bounds="clip") 
dec_iso.fit(true_train.time_spent_norm, true_train.accuracy)

lie_train["TRUE_calibrated_decision_time"] = dec_iso.predict(lie_train.decision_time)
raw_logloss  = log_loss(lie_train.correct_chosen, lie_train.decision_time)
cal_logloss  = log_loss(lie_train.correct_chosen, lie_train.TRUE_calibrated_decision_time)
print(f"raw LL={raw_logloss:.3f} → calibrated LL={cal_logloss:.3f}")

raw LL=1.027 → calibrated LL=0.969


In [49]:
lie_train.to_csv("/mnt/efs/shivamsinghal/reliability-aware-preference-learning/RAPL/datasets/scale_LIE/final_scale_lie_train.csv")

In [51]:
import glob
paths = glob.glob("/mnt/efs/shivamsinghal/reliability-aware-preference-learning/RAPL/datasets/scale_LIE/folds/*.csv")

In [ ]:
for path in paths:
    df = pd.read_csv(path)
    df["TRUE_calibrated_confidence"] = conf_iso.predict(df.confidence_scores)
    df["TRUE_calibrated_decision_time"] = dec_iso.predict(df.decision_time)
    df.to_csv(path, index=False)
    